In [1]:
import numpy as np
import pandas as pd
a = pd.read_excel("newdatajan2022.xls", usecols=np.r_[3,8,12,14,15,20,23,24,27,32,41,43,44,46,48,49,66])

In [2]:
a

,V4- Age 1-18-24 yr 2- 25-30 yrs 3- 31-35yrs 4-> 35 yrs,V2 - Time of diag 1- DURING PRESENT PREG 2- BEFORE PRESENT PREG,"V2 -NYHA CLASS III, IV- 1 I, II- 2",V2- ACQ-1 CONG-2,V2 - PROSTHETIC VALVE 1-YES 2-NO,"V2- mWHO >II- 1 I,II 2",V2 - LVEF <55 1-YES 2-NO,V2- LVEF ≤45 1-YES 2-NO,"V2 - PHT TRPG>40,RVSP>45 1-YES 2-NO",V2- MVOA ≤ 1.5 1-YES 2-NO,2V -ISOLATED VALVULAR AR/MR,V2- ISOLATED SIGNIFICANT AS 1-YES 2-NO,V2 - ISOLATED LVOTO SEVERITY 1- YES 2-NO,V2 -SIGNIFICANT LVOTO/RVOTO 1-YES 2-NO,V2 -PRIMI 1-YES 2- NO,V2 -NULLIPARA 1-YES 2-NO,V2- AFE -1 No AFE -2
0,1,1,2,2,2,1,2,2,2,2,2,2,2,2,2,1,1
1,2,2,2,2,2,1,2,2,2,2,2,2,2,2,1,1,2
2,2,2,2,1,2,1,2,2,2,2,2,2,2,2,2,2,2
3,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1
4,1,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2
1025,1,2,1,1,1,1,2,2,2,2,2,2,2,2,1,1,1
1026,2,1,1,1,2,1,1,1,1,2,2,2,2,2,2,2,2
1027,1,1,1,1,2,1,1,1,2,2,2,2,2,2,2,2,2


In [3]:
a.columns


Index(['V4-   Age                           1-18-24 yr               2- 25-30 yrs            3- 31-35yrs             4-> 35 yrs',
       ' V2 - Time of diag                        1- DURING PRESENT  PREG   2- BEFORE PRESENT PREG',
       'V2 -NYHA CLASS        III, IV- 1                    I, II- 2',
       'V2- ACQ-1                                    CONG-2      ',
       'V2 - PROSTHETIC VALVE                     1-YES                      2-NO',
       'V2- mWHO         >II-  1                    I,II    2',
       'V2 - LVEF <55          1-YES                 2-NO',
       'V2- LVEF ≤45       1-YES             2-NO',
       'V2 - PHT        TRPG>40,RVSP>45                          1-YES                       2-NO',
       'V2- MVOA ≤ 1.5           1-YES                       2-NO',
       '2V -ISOLATED  VALVULAR   AR/MR',
       'V2- ISOLATED SIGNIFICANT AS                       1-YES                 2-NO',
       'V2 - ISOLATED LVOTO SEVERITY                 1- YES               

In [17]:
X = a.drop('V2-  AFE -1                   No AFE -2',axis=1)
y = a['V2-  AFE -1                   No AFE -2']

In [18]:
import sklearn as sk
#from sklearn.datasets import make_classification
from sklearn.ensemble import AdaBoostClassifier
#from imblearn.over_sampling import RandomOverSampler

In [20]:
#from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=4, stratify = y)

model_ad = AdaBoostClassifier()
model_ad.fit(X_train , y_train)
y_pred = model_ad.predict(X_test)

print(classification_report(y_test, y_pred))
print(accuracy_score(y_test,y_pred))
#matrix = 
print(confusion_matrix(y_test,y_pred))

#print('Confusion matrix : \n',matrix)

              precision    recall  f1-score   support

           1       0.54      0.20      0.29        69
           2       0.69      0.91      0.79       137

    accuracy                           0.67       206
   macro avg       0.62      0.56      0.54       206
weighted avg       0.64      0.67      0.62       206

0.6747572815533981
[[ 14  55]
 [ 12 125]]


In [16]:
sum(y==2)

682

In [18]:
a.to_csv("filenewdatababyotc.csv")

In [23]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,12 hours 24 mins
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.3
H2O_cluster_version_age:,1 year and 1 month !!!
H2O_cluster_name:,H2O_from_python_josep_4kcqsf
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.851 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [20]:
a = h2o.import_file("filenewdatababyotc.csv")

train, test = a.split_frame(ratios=[.8])

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [22]:
# Identify predictors and response
x = train.columns
y = 'V2-  AFE -1                   No AFE -2'
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)


aml.leader

AutoML progress: |
22:04:38.892: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid__1_AutoML_20220124_220438_model_4


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,30.0,30.0,12244.0,4.0,13.0,9.3,6.0,46.0,27.8




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.15211610142910115
RMSE: 0.3900206423115335
LogLoss: 0.47413981021666873
Mean Per-Class Error: 0.19833152294914957
AUC: 0.8792496238673854
AUCPR: 0.9357683486955813
Gini: 0.7584992477347707

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5776643120993543: 


,,1,2,Error,Rate
0,1,166.0,103.0,0.3829,(103.0/269.0)
1,2,45.0,506.0,0.0817,(45.0/551.0)
2,Total,211.0,609.0,0.1805,(148.0/820.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.69756097,0.025725638,0.70731705,0.6768293,0.7256098,0.66463417,0.7134146
1,auc,0.65018165,0.037984997,0.6787431,0.69358975,0.5963522,0.6389572,0.643266
2,aucpr,0.78464323,0.025559288,0.7998983,0.8159864,0.78669345,0.75006086,0.7705772
3,err,0.30243903,0.025725638,0.29268292,0.32317072,0.27439025,0.33536586,0.28658536
4,err_count,49.6,4.2190046,48.0,53.0,45.0,55.0,47.0
5,f0point5,0.73711556,0.025957098,0.74862635,0.71132594,0.7677165,0.7086093,0.7492997
6,f1,0.81381756,0.018468091,0.81954885,0.7953668,0.83870965,0.795539,0.8199234
7,f2,0.90868753,0.008835958,0.90531564,0.90192646,0.9241706,0.90677965,0.90524536
8,lift_top_group,1.3445032,0.3492547,0.73214287,1.5769231,1.3898305,1.5327103,1.4909091
9,logloss,0.60396856,0.014481593,0.5862679,0.6037132,0.5975681,0.62583965,0.6064539



See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-01-24 22:04:50,1.168 sec,0.0,0.469503,0.632788,0.500000,0.671951,1.000000,0.328049
1,,2022-01-24 22:04:50,1.180 sec,5.0,0.449592,0.591845,0.772310,0.869209,1.488203,0.268293
2,,2022-01-24 22:04:50,1.192 sec,10.0,0.428004,0.548538,0.825029,0.902483,1.488203,0.226829
3,,2022-01-24 22:04:50,1.200 sec,15.0,0.424323,0.541149,0.816306,0.897343,1.488203,0.229268
4,,2022-01-24 22:04:50,1.214 sec,20.0,0.408230,0.509616,0.850788,0.917718,1.488203,0.200000
5,,2022-01-24 22:04:50,1.230 sec,25.0,0.398015,0.490003,0.869312,0.928760,1.488203,0.185366
6,,2022-01-24 22:04:50,1.245 sec,30.0,0.390021,0.474140,0.879250,0.935768,1.488203,0.180488



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,C1,193.972977,1.000000,0.621937
1,"V2 - PHT TRPG>40,RVSP>45 1-YES ...",17.827549,0.091907,0.057161
2,V4- Age 1-18-24 yr 2- 25...,14.158564,0.072992,0.045397
3,V2 - PROSTHETIC VALVE 1-YES ...,13.336250,0.068753,0.042760
4,V2 -PRIMI 1-YES 2- NO,11.326597,0.058393,0.036317
5,V2- MVOA ÃƒÆ’Ã‚Â¢ÃƒÂ¢Ã¢â€šÂ¬Ã‚Â°Ãƒâ€šÃ‚Â¤ 1.5 1-YES ...,9.618866,0.049589,0.030841
6,"V2- mWHO >II- 1 I,II 2",8.565146,0.044156,0.027463
7,V2 - Time of diag 1- DURING PRESENT PREG ...,8.552720,0.044092,0.027423
8,"V2 -NYHA CLASS III, IV- 1 I, II- 2",8.050931,0.041505,0.025814
9,V2- ACQ-1 CONG-2,7.569145,0.039022,0.024269


In [ ]:
x[17]

In [26]:
a = h2o.import_file("filenewdatababyotc.csv")
x = train.columns
y = 'V2-  AFE -1                   No AFE -2'
x.remove(y)
a[y] = a[y].asfactor()
level1,test1,level2,test2,final = a.split_frame([0.4,0.1,0.05,0.25],seed=1234)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [27]:
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.xgboost import H2OXGBoostEstimator
from h2o.estimators.naive_bayes import H2ONaiveBayesEstimator
nfolds=3

In [28]:
from collections import OrderedDict
models = OrderedDict([
                        ('GBM', H2OGradientBoostingEstimator(model_id="GBM",nfolds=nfolds, fold_assignment="Modulo",\
                                    keep_cross_validation_predictions=True,\
                                    ntrees=1000,max_depth=5,learn_rate=0.2)),
                        ('RF', H2ORandomForestEstimator(model_id="RF",nfolds=nfolds, fold_assignment="Modulo",\
                                    keep_cross_validation_predictions=True,\
                                    ntrees=100,mtries=7,max_depth=30)),
                        ('DL', H2ODeepLearningEstimator(model_id="DL",nfolds=nfolds, fold_assignment="Modulo",\
                                    keep_cross_validation_predictions=True)),\
                        #('XGB', H2OXGBoostEstimator(model_id="XGB",nfolds=nfolds, fold_assignment="Modulo",\
                        #            keep_cross_validation_predictions=True)),\
                        ('NB', H2ONaiveBayesEstimator(model_id="NB",nfolds=nfolds, fold_assignment="Modulo",\
                                    keep_cross_validation_predictions=True))
                    ])

In [29]:
# Training each model in the dictionary
for model in models.values():
    model.train(x=x,y=y,training_frame=level1, validation_frame=test1)

# Stacked Ensemble Model
stack = H2OStackedEnsembleEstimator(model_id="my_ensemble", base_models=list(models.keys()))
# Stacked Model Traning
stack.train(x=x,y=y,training_frame=level1,validation_frame=test1)

#Calculating accuracy of each model
models_acc = []
for model in models.values():
    models_acc.append(model.accuracy(valid=True)[0][1])

Ensemble_acc=stack.accuracy(valid=True)[0][1]

print(models_acc,Ensemble_acc)

gbm Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
naivebayes Model Build progress: |████████████████████████████████████████| 100%
stackedensemble Model Build progress: |███████████████████████████████████| 100%
[0.7589285714285714, 0.75, 0.7321428571428571, 0.7589285714285714] 0.7767857142857143


In [30]:
ll=level2[y].as_data_frame()

In [31]:
pd.get_dummies(ll, columns=[y])

,V2- AFE -1 No AFE -2_1,V2- AFE -1 No AFE -2_2
0,0,1
1,0,1
2,0,1
3,0,1
4,1,0
5,0,1
6,0,1
7,0,1
8,0,1
9,0,1


In [32]:
#block is added by Joel Rajakumar to train the model selector
import pandas as pd
import numpy as np
def score(targets,predictions):
    #Cross Entropy
    return -np.sum(targets*np.log(predictions).values, axis=1)

level2scores = pd.DataFrame()
test2scores = pd.DataFrame()

#One-hot encoded truth values
level2truths = pd.get_dummies(level2[y].as_data_frame(), columns=[y])
test2truths = pd.get_dummies(test2[y].as_data_frame(), columns=[y])

for model in models.values():
    #Get predictions
    level2predictions = model.predict(level2).drop('predict').as_data_frame()
    test2predictions = model.predict(test2).drop('predict').as_data_frame()
    
    #Get scores
    level2scores = pd.concat([level2scores, score(level2truths,level2predictions)], axis=1)
    test2scores = pd.concat([test2scores, score(test2truths,test2predictions)], axis=1)

#Find the winner (1st place)
level2winners = np.argmin(level2scores.values, axis=1)
test2winners = np.argmin(test2scores.values, axis=1)

level2['winners'] = h2o.H2OFrame([list(models.keys())[x] for x in level2winners])
test2['winners'] = h2o.H2OFrame([list(models.keys())[x] for x in test2winners])


gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


C:\Users\josep\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


deeplearning prediction progress: |███████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
naivebayes prediction progress: |█████████████████████████████████████████| 100%
naivebayes prediction progress: |█████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [33]:
#Training Model Selector
response='winners'
predictors=level2.columns
predictors.remove(y)
predictors.remove(response)

modelselector=H2OGradientBoostingEstimator(model_id="modelselector",nfolds=nfolds, fold_assignment="Modulo",\
                                    keep_cross_validation_predictions=True,\
                                   ntrees=1000,max_depth=5,learn_rate=0.2)

modelselector.train(x=predictors,y=response,training_frame=level2, validation_frame=test2)

gbm Model Build progress: |█████████████████████████████████████████████████████████| 100%


In [34]:
def predict(data):
    #apply model selector
    data['winner'] = modelselector.predict(data)['predict']
    all_data = []
    for name, model in models.items():
        #Slice data out for each model
        model_data = data[data['winner']==name]
        
        #Apply each model to respective slice
        model_data = model_data.cbind(model.predict(model_data))
        all_data.append(model_data)
        
    #concatenate it all
    stuff = model_data[model_data['C1']==-1]
    stuff = stuff.concat(all_data,axis=0)
    
    #merge it with the original data
    data = stuff.merge(data) #This step should redundant and can be deleted
    return stuff

In [35]:
results = predict(final)

gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%
naivebayes prediction progress: |█████████████████████████████████████████| 100%


In [36]:
#Accuracy
b = results[y] == results['predict']
print("Accuracy with dynamic model selection: {0:.3f}%\n".format(b.sum()/b.shape[0]*100))
print("Accuracy with static model selection : {0:.3f}%\n".format(Ensemble_acc*100))

Accuracy with dynamic model selection: 63.768%

Accuracy with static model selection : 77.679%

